In [1]:
from data import * 
from model import *
data = Dataset()
data.to_torch()
data.train_val_test_split()
model = AttentionModel()
model.train_model(data=data)

Epoch  1 | Train Loss: 0.6925 | Val Loss: 0.6902
Epoch  2 | Train Loss: 0.6835 | Val Loss: 0.6829
Epoch  3 | Train Loss: 0.6755 | Val Loss: 0.6783
Epoch  4 | Train Loss: 0.6678 | Val Loss: 0.6761
Epoch  5 | Train Loss: 0.6654 | Val Loss: 0.6663
Epoch  6 | Train Loss: 0.6566 | Val Loss: 0.6634
Epoch  7 | Train Loss: 0.6443 | Val Loss: 0.6471
Epoch  8 | Train Loss: 0.6391 | Val Loss: 0.6446
Epoch  9 | Train Loss: 0.6289 | Val Loss: 0.6277
Epoch 10 | Train Loss: 0.6051 | Val Loss: 0.6139
Epoch 11 | Train Loss: 0.5508 | Val Loss: 0.5643
Epoch 12 | Train Loss: 0.5551 | Val Loss: 0.5204
Epoch 13 | Train Loss: 0.4519 | Val Loss: 0.3916
Epoch 14 | Train Loss: 0.4098 | Val Loss: 0.3853
Epoch 15 | Train Loss: 0.3471 | Val Loss: 0.3499
Epoch 16 | Train Loss: 0.3314 | Val Loss: 0.4724
Epoch 17 | Train Loss: 0.4503 | Val Loss: 0.3849
Epoch 18 | Train Loss: 0.3231 | Val Loss: 0.3037
Epoch 19 | Train Loss: 0.3356 | Val Loss: 0.3402
Epoch 20 | Train Loss: 0.3139 | Val Loss: 0.3073
Epoch 21 | Train Los

In [46]:
import matplotlib.pyplot as plt
import numpy as np
import torch

def plot_attention(sample_X, attention_weights, title="Attention over Time", save_path=None):
    """
    Visualizes attention weights over the input time series.

    Args:
        sample_X: torch.Tensor or np.array of shape (seq_len, 1)
        attention_weights: torch.Tensor or np.array of shape (seq_len,)
        title: plot title
        save_path: optional path to save the figure
    """
    if isinstance(sample_X, torch.Tensor):
        sample_X = sample_X.detach().cpu().numpy().squeeze()
    if isinstance(attention_weights, torch.Tensor):
        attention_weights = attention_weights.detach().cpu().numpy().squeeze()

    seq_len = len(sample_X)
    time_steps = np.arange(seq_len)

    fig, ax1 = plt.subplots(figsize=(10, 4))

    ax1.set_title(title)
    ax1.set_xlabel("Time Step")
    ax1.set_ylabel("Signal", color='blue')
    ax1.plot(time_steps, sample_X, color='blue', linewidth=2)
    ax1.tick_params(axis='y', labelcolor='blue')

    ax2 = ax1.twinx()
    ax2.set_ylabel("Attention", color='red')
    ax2.plot(time_steps, attention_weights*10, color='red', linestyle='--', alpha=0.7)
    ax2.tick_params(axis='y', labelcolor='red')
    ax2.set_ylim(0, 1.1)

    plt.tight_layout()
    if save_path:
        plt.savefig(save_path)
    plt.show()


In [1]:
model.eval()
k = 0
with torch.no_grad():
    for batch_X, batch_Y in data.val_loader:
        batch_X = batch_X.to('cpu')
        outputs, attn_weights = model(batch_X)

        # Let's plot the first sample in the batch
        plot_attention(batch_X[k], attn_weights[k], title="Example Attention")
        break


NameError: name 'model' is not defined

In [42]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score

def evaluate_model(model, dataloader, device="cpu"):
    model.eval()

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch_X, batch_Y in dataloader:
            batch_X = batch_X.to(device)
            batch_Y = batch_Y.to(device)

            outputs, _ = model(batch_X)
            preds = (outputs > 0.5).float()  # binary thresholding

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(batch_Y.cpu().numpy())

    all_preds = np.array(all_preds).flatten()
    all_labels = np.array(all_labels).flatten()

    acc = accuracy_score(all_labels, all_preds)
    prec = precision_score(all_labels, all_preds, zero_division=0)
    rec = recall_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    auc = roc_auc_score(all_labels, all_preds)

    print(f"Accuracy      : {acc:.4f}")
    print(f"Precision     : {prec:.4f}")
    print(f"Recall        : {rec:.4f}")
    print(f"F1 Score      : {f1:.4f}")
    print(f"ROC AUC Score : {auc:.4f}")
    print("\nConfusion Matrix:")
    print(confusion_matrix(all_labels, all_preds))


In [44]:
test_loader = data.test_loader
evaluate_model(model, test_loader, device='cpu')


Accuracy      : 0.9710
Precision     : 0.9936
Recall        : 0.9472
F1 Score      : 0.9699
ROC AUC Score : 0.9706

Confusion Matrix:
[[1009    6]
 [  52  933]]
